In [1]:
import tensorflow as tf
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import math
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense

In [2]:
!wget --no-check-certificate \https://www.kaggle.com/mczielinski/bitcoin-historical-data

--2021-05-23 10:03:50--  https://www.kaggle.com/mczielinski/bitcoin-historical-data
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘bitcoin-historical-data’

bitcoin-historical-     [ <=>                ]  62.73K  --.-KB/s    in 0.06s   

2021-05-23 10:03:50 (981 KB/s) - ‘bitcoin-historical-data’ saved [64232]



In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
def preprocess_data(file_path):
  """function that preprocesses the data from the dataset"""
  df = pd.read_csv(/content/drive/MyDrive/Colab Notebooks/Datasets/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv)
  # Remove all NaN-containing entries
  df = df.dropna()
  # Reformat the timestamp
  df['Timestamp'] = pd.to_datetime(df['Timestamp'],
  infer_datetime_format=True, unit='s')
  # Use the timestamp as index column
  df = df.set_index('Timestamp')
  # Narrow the scope of dataframe columns to work with
  df = df.drop(['Low', 'High', 'Volume_(BTC)', 'Weighted_Price'], 
               axis=1)
  # Reorder columns
  df = df.reindex(columns=['Open', 'Close', 'Volume_(Currency)'])
  # Work on a 1hour window
  df['Open'] = df['Open'].resample('1H').first()
  df['Close'] = df['Close'].resample('1H').last()
  df['Volume_(Currency)'] = df['Volume_(Currency)'].resample('1H').sum()
  # Remove all NaN-containing entries
  df = df.dropna()
  # Remove the first half of the dataframe (given data sparsity)
  df = df.iloc[-int((df.shape[0]/2)):]
  print(df.head(10))
  print('=======================')
  # Create the dataset (np.ndarray of "df.shape")
  dataset = df.values
  print(dataset[:10])
  print(dataset.shape)
  print('=======================')
  # Standardize the dataset
  mean = np.mean(dataset, axis=0)
  stddev = np.std(dataset, axis=0)
  dataset = (dataset - mean) / stddev
  print(dataset[:10])
  print(dataset.shape)
  print('=======================')
  def split_sequence(sequence, n_steps):
    """function that splits a dataset sequence into input data and
    labels"""
    X, Y = [], []
    for i in range(sequence.shape[0]):
      if (i + n_steps) >= sequence.shape[0]:
        break
      # Divide sequence between data (input) and labels (output)
      seq_X, seq_Y = sequence[i: i + n_steps],sequence[i + n_steps, -2]
      X.append(seq_X)
      Y.append(seq_Y)
    return np.array(X), np.array(Y)
  # Create training and validation datasets
  dataset_size = dataset.shape[0]
  x_train, y_train = split_sequence(dataset[0: math.ceil(0.7 *
                                    dataset_size)], 24)
  x_val, y_val = split_sequence(dataset[math.floor(0.7 *
                                dataset_size):], 24)
  return dataset, df, x_train, y_train, x_val, y_val

file_path = './bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv'
dataset, df, x_train, y_train, x_val, y_val = preprocess_data(file_path)
print("dataset.shape: {}".format(dataset.shape))
print("df.shape: {}".format(df.shape))
print("x_train.shape: {}".format(x_train.shape))
print("y_train.shape: {}".format(y_train.shape))
print("x_val.shape: {}".format(x_val.shape))
print("y_val.shape: {}".format(y_val.shape))
print('=======================')

SyntaxError: ignored